## Exploratory Data Analysis

In [52]:
# Import Libraries:

import pandas as pd
import numpy as py

In [53]:
# Import JSON 

import json
with open('/Users/jnate/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict:
login.keys()

dict_keys(['api-key'])

Each year should be saved as a separate .csv.gz file

In [54]:
# Load API key:

import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [55]:
# Make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

In [56]:
# Movie ofbjects have a .info dictionary
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/y9wuhlrqSHvhTLNVNwKMKe6HZzY.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 79.658,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 174,
   'logo_path': '/IuAlhI9eVC9Z8UQWOIDdWRKSEJ.png'

In [57]:
info['budget']

63000000

In [58]:
info['revenue']

463517383

In [59]:
# info() includes "imbdb_id" which we can use to match with 
# our existing data frame of movies:

info['imdb_id']

'tt0133093'

Your stakeholder wants you to extract the budget, revenue, and MPAA Rating (G/PG/PG-13/R), which is also called "Certification".

As a proof-of-concept, they requested you perform a test extraction of movies that started in 2000 or 2001

In [60]:
# load csv files from part I:

basics = pd.read_csv('C:\\Users\\jnate\\OneDrive\\Documents\\Data Enrichment\\Project-3\\Data\\title_basics.csv.gz')

basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001-01-01,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020-01-01,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018-01-01,NaN,122,Drama
3,tt0079644,movie,November 1828,November 1828,0,2001-01-01,NaN,140,"Drama,War"
4,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005-01-01,NaN,100,"Comedy,Horror,Sci-Fi"


In [61]:
# Save API calls to a previously created folder called Dat:

import os
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['akas.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'ratings.csv.gz',
 'title_basics.csv.gz',
 'tmbd_api_results_2000.json',
 'tmbd_api_results_2001.json']

In [62]:
# Function to append new results to the existing JSON file:

def write_json(new_data, filename):
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""
    with open(filename, 'r+') as file:
        # Load existing data into a dictionary:
        file_data = json.load(file)
        
        # Choose extend or append:
        if (type(new_data)== list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


# # Defining our extraction focus: movies from 2000 and 2001 only:

# YEARS_TO_GET = [2000,2001]

# # Creating a progress bar to ensure that the program is working properly:

# from tqdm.notebook import tqdm_notebook
# import time

# # OUTER LOOP
# for year in tqdm_notebook(years,desc='years',position=0):
    
#     # Defining the JSON file that will store results for each year:
#     JSON_FILE = f'{FOLDER}tmbd_api_results_{year}.json'
    
#     # Check to see if the file is existing: 
#     file_exist = os.path.isfile(JSON_FILE)
    
#     # Create the file and save an empty dictionary 
#     if file_exist == False:
        
#     # Save empty dictionary with imdb IDs
#         with open(JSON_FILE, 'w') as f:
#             json.dump([{'imdb_id' : 0}], f)
            
#             # Breaking up dataset by year. Each new year gets a separate dataframe:

#             # Saving the latest year as the current df
#             df = basics.loc[basics['startYear']==year].copy()

#             # Saving movie IDs to a list:
#             movie_ids = df['tconst'].copy()

#             # Load existing data from json into a dataframe called previous df:
#             previous_df = pd.read_json(JSON_FILE)

#             # Removing redundant movie IDs
#             movie_ids_of_interest = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

#             # Get index and movie ID from list
#             # INNER Loop
#             for movie_id in tqdm_notebook(movie_ids_of_interest,
#                               desc=f'Movies from {year}',
#                               position =1, 
#                               leave=True):
#                 # Attempt to retrieve then data fro the movie ID:
#                 try:
#                     temp = get_movie_with_rating(movie_id) # Premade function
        
#                 # Append / add results to existing file using a pre-made function:
#                     write_json(temp, JSON_FILE)
        
#                 # Short 20ms sleep to prevent overwhelming server
#                     time.sleep(0.02)
#                 # If it fails - create a dictionary with just the ID and None for certification:
#                 except Exception as e:
#                     continue
#     # Save the year's results as a csv.gz file:
#     final_year_df = pd.read_json(JSON_FILE)
#     final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{year}.csv.gz", compression = "gzip", 
#                      index = False)

In [63]:
YEARS_TO_GET = [2000,2001]


# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',
                          position=0):
    #Defining the JSON file to store results for year
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)


    # If it does not exist: create it
    if file_exists == False:
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)



    #Saving new year as the current df
    df = basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()


    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)


    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]


    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)

        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue


    
    final_year_df = pd.read_json(JSON_FILE)
    
    csv_fname = f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz"
    final_year_df.to_csv(csv_fname, compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000: 0it [00:00, ?it/s]

Movies from 2001: 0it [00:00, ?it/s]

### Defining & Filtering IDs to Call:
    

### Iterate through Movie IDs to make API calls